<a href="https://colab.research.google.com/github/kquacken/module5_intro_to_pyspark/blob/main/5_6_stack_and_unstack_in_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Setting up Colab

To run this notebook on [Google's Colab](https://colab.research.google.com), you will need to perform the following steps.

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 64 kB/s 
     |████████████████████████████████| 199 kB 69.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=c0f3cb7abfef2e9dc6681588d399524c0fddb639af439197f37cd5aa8b2a2f37
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!pip install composable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.7 MB/s 
  Attempting uninstall: toolz
    Found existing installation: toolz 0.12.0
    Uninstalling toolz-0.12.0:
      Successfully uninstalled toolz-0.12.0


Step 2. Download and unzip the data

Next, the easiest way to access the data from the module is to download and unzip.

In [3]:
!wget https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/data.zip

--2022-10-25 00:30:43--  https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/data.zip [following]
--2022-10-25 00:30:43--  https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977210 (14M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  14.28M  --.-KB/s    in 0.09s   

2022-10-25 00:30:43 (167 MB/s) - ‘data.zip’ saved [14977210/14977210]



In [4]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: __MACOSX/._data         
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/Rochester_temps_2019.xlsx  
  inflating: __MACOSX/data/._Rochester_temps_2019.xlsx  
  inflating: data/uber-raw-data-jun14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-jun14-sample.csv  
  inflating: data/heroes_information.csv  
  inflating: __MACOSX/data/._heroes_information.csv  
  inflating: data/employee.csv       
  inflating: __MACOSX/data/._employee.csv  
  inflating: data/uber-raw-data-apr14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-apr14-sample.csv  
   creating: data/baseball/
  inflating: __MACOSX/data/._baseball  
  inflating: data/uber-raw-data-may14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-may14-sample.csv  
  inflating: data/auto_sales_apr.csv  
  inflating: __MACOSX/data/._auto_sales_apr.csv  
  inflating: data/uber-raw-data-sep14-sample.csv  
  inflating: __M

In [5]:
!wget https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/more_pyspark.py

--2022-10-25 00:30:44--  https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/more_pyspark.py
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/more_pyspark.py [following]
--2022-10-25 00:30:44--  https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/more_pyspark.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2786 (2.7K) [text/plain]
Saving to: ‘more_pyspark.py’

more_pyspark.py     100%[===================>]   2.72K  --.-KB/s    in 0s      

2022-10-25 00:30:44 (37.5 MB/s) - ‘more_pyspark.py’ saved [2786/2786]



# Stacking and Unstacking data in `pyspark`

## Welcome back to developer's corner

While `pyspark` doesn't provide explicit methods for these two actions, we will be able to accomplish each operation using a combination of methods.  In this lecture, we will 

1. Build up expressions for stacking and unstacking a `pyspark df` 
2. Create helper functions for each operation to automate these processes.

#### `pyspark` setup

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ops').config('spark.driver.host', 'localhost').getOrCreate()

In [7]:
from more_pyspark import to_pandas
sales = spark.read.csv("./data/auto_sales.csv",  
                       header=True, 
                       inferSchema=True)
sales.collect() >> to_pandas

,Salesperson,Compact,Sedan,SUV,Truck
0,Ann,22,18,15.0,12
1,Bob,19,12,17.0,20
2,Doug,20,13,NaN,20
3,Yolanda,19,8,32.0,15
4,Xerxes,12,23,18.0,9


## Stacking data in `pyspark`

Two step process

1. Combine the columns to be stacked into an array
2. Explode the array

[Source](https://www.mien.in/2018/03/25/reshaping-dataframe-using-pivot-and-melt-in-apache-spark-and-pandas/)

#### A helper function that combines multiple column entries into a column array

In [8]:
from pyspark.sql.functions import array, explode, struct, lit, col
make_array = lambda var_name, val_name, cols: (array(*(struct(lit(c).alias(var_name), 
                                                              col(c).alias(val_name))
                                                       for c in cols)))

sales_cols = ['Compact','Sedan', 'SUV', 'Truck']
make_array('car_type', 'qtr_sales', sales_cols)

Column<'array(struct(Compact AS car_type, Compact AS qtr_sales), struct(Sedan AS car_type, Sedan AS qtr_sales), struct(SUV AS car_type, SUV AS qtr_sales), struct(Truck AS car_type, Truck AS qtr_sales))'>

#### Testing `make_array`

In [9]:
from pyspark.sql.functions import array, explode, struct, lit, col, collect_list
(sales
 .withColumn('sales', 
             make_array('car_type', 
                        'qtr_sales', 
                        sales_cols))
 .collect()) >> to_pandas

,Salesperson,Compact,Sedan,SUV,Truck,sales
0,Ann,22,18,15.0,12,"[(Compact, 22), (Sedan, 18), (SUV, 15), (Truck..."
1,Bob,19,12,17.0,20,"[(Compact, 19), (Sedan, 12), (SUV, 17), (Truck..."
2,Doug,20,13,NaN,20,"[(Compact, 20), (Sedan, 13), (SUV, None), (Tru..."
3,Yolanda,19,8,32.0,15,"[(Compact, 19), (Sedan, 8), (SUV, 32), (Truck,..."
4,Xerxes,12,23,18.0,9,"[(Compact, 12), (Sedan, 23), (SUV, 18), (Truck..."


#### `explode` the contents to the array column to stack entries

In [10]:
t = (sales
    .withColumn('sales', 
                make_array('car_type', 
                           'qtr_sales', 
                           sales_cols))
    .withColumn("vars_and_vals", explode(col('sales'))))
t.take(3) >> to_pandas

,Salesperson,Compact,Sedan,SUV,Truck,sales,vars_and_vals
0,Ann,22,18,15,12,"[(Compact, 22), (Sedan, 18), (SUV, 15), (Truck...","(Compact, 22)"
1,Ann,22,18,15,12,"[(Compact, 22), (Sedan, 18), (SUV, 15), (Truck...","(Sedan, 18)"
2,Ann,22,18,15,12,"[(Compact, 22), (Sedan, 18), (SUV, 15), (Truck...","(SUV, 15)"


#### Checking out the schema of the result

Note that the exploded entries are named `'car_type'` and `'qtr_sales'`

In [11]:
t.printSchema()

root
 |-- Salesperson: string (nullable = true)
 |-- Compact: integer (nullable = true)
 |-- Sedan: integer (nullable = true)
 |-- SUV: integer (nullable = true)
 |-- Truck: integer (nullable = true)
 |-- sales: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- car_type: string (nullable = false)
 |    |    |-- qtr_sales: integer (nullable = true)
 |-- vars_and_vals: struct (nullable = false)
 |    |-- car_type: string (nullable = false)
 |    |-- qtr_sales: integer (nullable = true)



#### Testing extraction of tuple entries

In [12]:
t = (t
 .withColumn('types', t['vars_and_vals'].getItem("car_type"))
 .withColumn('sales', t['vars_and_vals'].getItem("qtr_sales")))
t.take(3) >> to_pandas

,Salesperson,Compact,Sedan,SUV,Truck,sales,vars_and_vals,types
0,Ann,22,18,15,12,22,"(Compact, 22)",Compact
1,Ann,22,18,15,12,18,"(Sedan, 18)",Sedan
2,Ann,22,18,15,12,15,"(SUV, 15)",SUV


#### Dropping extra columns


In [13]:
t = (t
 .drop(*sales_cols)
 .drop('vars_and_vals'))
t.take(3) >> to_pandas

,Salesperson,sales,types
0,Ann,22,Compact
1,Ann,18,Sedan
2,Ann,15,SUV


####  Putting it all together with generic names

In [14]:
cols_to_stack = sales_cols
var_lbl = 'car_type'
val_lbl = 'qtr_sales'
(sales
 .withColumn('var_val_array', 
             make_array(var_lbl, 
                        val_lbl, 
                        cols_to_stack))
 .withColumn("vars_and_vals", explode(col('var_val_array')))
 .withColumn(var_lbl, col("vars_and_vals").getItem(var_lbl))
 .withColumn(val_lbl, col("vars_and_vals").getItem(val_lbl))
 .drop(*(cols_to_stack + ['var_val_array', "vars_and_vals"]))
 .collect()) >> to_pandas

,Salesperson,car_type,qtr_sales
0,Ann,Compact,22.0
1,Ann,Sedan,18.0
2,Ann,SUV,15.0
3,Ann,Truck,12.0
4,Bob,Compact,19.0
5,Bob,Sedan,12.0
6,Bob,SUV,17.0
7,Bob,Truck,20.0
8,Doug,Compact,20.0
9,Doug,Sedan,13.0


#### Building a `gather` functions

In [15]:
from composable import pipeable

cols_to_stack = sales_cols
var_lbl = 'car_type'
val_lbl = 'qtr_sales'

@pipeable
def gather(var_lbl, val_lbl, cols_to_stack, df):
    make_array = lambda var_name, val_name, cols: (array(*(struct(lit(c).alias(var_name), 
                                                                  col(c).alias(val_name))
                                                           for c in cols)))
    return (df
            .withColumn('var_val_array', 
                        make_array(var_lbl, 
                                   val_lbl, 
                                   cols_to_stack))
            .withColumn("vars_and_vals", explode(col('var_val_array')))
            .withColumn(var_lbl, col("vars_and_vals").getItem(var_lbl))
            .withColumn(val_lbl, col("vars_and_vals").getItem(val_lbl))
            .drop(*(cols_to_stack + ['var_val_array', "vars_and_vals"])))
sales_stacked = sales >> gather('car_type', 'qtr_sales', sales_cols)
sales_stacked.collect() >> to_pandas

,Salesperson,car_type,qtr_sales
0,Ann,Compact,22.0
1,Ann,Sedan,18.0
2,Ann,SUV,15.0
3,Ann,Truck,12.0
4,Bob,Compact,19.0
5,Bob,Sedan,12.0
6,Bob,SUV,17.0
7,Bob,Truck,20.0
8,Doug,Compact,20.0
9,Doug,Sedan,13.0


## Unstacking columns in `pyspark` with `group_by` and `pivot`

In [16]:
(sales_stacked
 .groupBy('Salesperson')
 .pivot('car_type')
 .sum('qtr_sales')
 .collect()) >> to_pandas

,Salesperson,Compact,SUV,Sedan,Truck
0,Yolanda,19,32.0,8,15
1,Doug,20,NaN,13,20
2,Xerxes,12,18.0,23,9
3,Ann,22,15.0,18,12
4,Bob,19,17.0,12,20


#### Creating a `spread` helper function

In [17]:
from pyspark.sql.functions import array, explode, struct, lit, col

@pipeable
def spread(val_col, var_col, group_by_col, df):
    return  (df
             .groupBy(group_by_col)
             .pivot(val_col)
             .sum(var_col))
sales_unstacked = sales_stacked >> spread('car_type', 'qtr_sales', 'Salesperson')
sales_unstacked.take(5) >> to_pandas

,Salesperson,Compact,SUV,Sedan,Truck
0,Yolanda,19,32.0,8,15
1,Doug,20,NaN,13,20
2,Xerxes,12,18.0,23,9
3,Ann,22,15.0,18,12
4,Bob,19,17.0,12,20


## <font color="red"> Exercise 3 </font>
    
**Task:** Load the `Rochester_temps_2019.csv` data, contains data weather data for Rochester, MN. that is available at the [DNR website](https://www.dnr.state.mn.us/climate/historical/lcd.html?loc=rst). Note that `SM` and `AV` stand for *sum* and *average*, respectively.

1. Identify that problem with the current format.
2. Use `gather` and `spread` to fix the issue.

In [29]:
min_max = spark.read.csv("./data/Rochester_temps_2019.csv", header=True, inferSchema=True)
min_max.take(10) >> to_pandas

,ELEMENT,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,MEAN DAILY MAXIMUM,19.7,20.3,33.9,54.6,63.5,76.0,81.7,76.7,74.7,52.1,34.6,29.2
1,HIGHEST DAILY MAXIMUM,44.0,41.0,61.0,83.0,86.0,87.0,90.0,85.0,89.0,71.0,47.0,42.0
2,MEAN DAILY MINIMUM,4.4,2.4,17.9,35.1,44.3,56.5,62.6,56.7,55.5,35.6,21.5,13.9
3,LOWEST DAILY MINIMUM,-30.0,-15.0,-17.0,24.0,33.0,43.0,52.0,48.0,46.0,19.0,0.0,-8.0
4,AVERAGE DRY BULB,12.1,11.3,25.9,44.8,53.9,66.3,72.2,66.7,65.1,43.9,28.1,21.5
5,MEAN WET BULB,11.3,NaN,NaN,NaN,48.2,59.9,67.3,61.1,60.6,NaN,26.4,21.6
6,MEAN DEW POINT,8.3,NaN,NaN,NaN,43.0,55.2,64.3,57.8,57.5,NaN,22.6,18.3
7,NUM DAYS W MAXIMUM >= 90°,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,NUM DAYS W MAXIMUM <= 32°,25.0,24.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,17.0
9,NUM DAYS W MINIMUM <= 32°,31.0,27.0,27.0,11.0,0.0,0.0,0.0,0.0,0.0,14.0,29.0,31.0


> Months are data and elements are variables

In [26]:
from composable import pipeable
sales_cols2 = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']
cols_to_stack = sales_cols2
var_lbl = 'Month'
val_lbl = 'temp'

@pipeable
def gather(var_lbl, val_lbl, cols_to_stack, df):
    make_array = lambda var_name, val_name, cols: (array(*(struct(lit(c).alias(var_name), 
                                                                  col(c).alias(val_name))
                                                           for c in cols)))
    return (df
            .withColumn('var_val_array', 
                        make_array(var_lbl, 
                                   val_lbl, 
                                   cols_to_stack))
            .withColumn("vars_and_vals", explode(col('var_val_array')))
            .withColumn(var_lbl, col("vars_and_vals").getItem(var_lbl))
            .withColumn(val_lbl, col("vars_and_vals").getItem(val_lbl))
            .drop(*(cols_to_stack + ['var_val_array', "vars_and_vals"])))
temp_stacked = min_max >> gather('Month', 'temp', sales_cols2)
temp_stacked.collect() >> to_pandas

,ELEMENT,Month,temp
0,MEAN DAILY MAXIMUM,JAN,19.7
1,MEAN DAILY MAXIMUM,FEB,20.3
2,MEAN DAILY MAXIMUM,MAR,33.9
3,MEAN DAILY MAXIMUM,APR,54.6
4,MEAN DAILY MAXIMUM,MAY,63.5
...,...,...,...
127,NUM DAYS W MINIMUM <= 0°,AUG,0.0
128,NUM DAYS W MINIMUM <= 0°,SEP,0.0
129,NUM DAYS W MINIMUM <= 0°,OCT,0.0
130,NUM DAYS W MINIMUM <= 0°,NOV,1.0


In [27]:
gather_temp = (temp_stacked
 .groupBy('Month')
 .pivot('ELEMENT')
 .sum('temp')
 .collect()) 
gather_temp >> to_pandas()

,Month,NUM DAYS W MINIMUM <= 0°,AVERAGE DRY BULB,HIGHEST DAILY MAXIMUM,LOWEST DAILY MINIMUM,MEAN DAILY MAXIMUM,MEAN DAILY MINIMUM,MEAN DEW POINT,MEAN WET BULB,NUM DAYS W MAXIMUM <= 32°,NUM DAYS W MINIMUM <= 32°,NUM DAYS W MAXIMUM >= 90°
0,APR,0.0,44.8,83.0,24.0,54.6,35.1,NaN,NaN,0.0,11.0,0.0
1,OCT,0.0,43.9,71.0,19.0,52.1,35.6,NaN,NaN,1.0,14.0,0.0
2,NOV,1.0,28.1,47.0,0.0,34.6,21.5,22.6,26.4,12.0,29.0,0.0
3,FEB,15.0,11.3,41.0,-15.0,20.3,2.4,NaN,NaN,24.0,27.0,0.0
4,SEP,0.0,65.1,89.0,46.0,74.7,55.5,57.5,60.6,0.0,0.0,0.0
5,JAN,11.0,12.1,44.0,-30.0,19.7,4.4,8.3,11.3,25.0,31.0,0.0
6,AUG,0.0,66.7,85.0,48.0,76.7,56.7,57.8,61.1,0.0,0.0,0.0
7,MAR,6.0,25.9,61.0,-17.0,33.9,17.9,NaN,NaN,11.0,27.0,0.0
8,DEC,6.0,21.5,42.0,-8.0,29.2,13.9,18.3,21.6,17.0,31.0,0.0
9,JUN,0.0,66.3,87.0,43.0,76.0,56.5,55.2,59.9,0.0,0.0,0.0
